In [15]:
import cv2
import keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.models import load_model
from keras.utils.generic_utils import CustomObjectScope

from models.unets import Unet2D
#from models.separable_unet import Separable_Unet2D
from models.deeplab import Deeplabv3, relu6, BilinearUpsampling, DepthwiseConv2D
from models.FCN import FCN_Vgg16_16s
from models.SegNet import SegNet

from utils.learning.metrics import dice_coef, precision, recall
from utils.BilinearUpSampling import BilinearUpSampling2D
from utils.io.data import load_data, save_results, save_rgb_results, save_history, load_test_images, DataGen

In [16]:
# settings
input_dim_x = 224
input_dim_y = 224
color_space = 'rgb'
path = './data/all_dog_wounds_noAugmentation/'#all_dog_wounds_noAugmentation#Medetec_foot_ulcer_224#Dog_wound_low_resolution#dog9_10_test
weight_file_name = '2021-05-05 02:22:24.668040.hdf5'
pred_save_path = '2021-05-05 02:22:24.668040/'

#data_gen = DataGen(path, split_ratio=0.0, x=input_dim_x, y=input_dim_y, color_space=color_space)
x_test, test_label_filenames_list = load_test_images(path)

In [17]:
# ### get unet model
#unet2d = Unet2D(n_filters=64, input_dim_x=input_dim_x, input_dim_y=input_dim_y, num_channels=3)
#model = unet2d.get_unet_model_yuanqing()
model = load_model('./merged_azh_dog1_8_history/' + weight_file_name,
                   custom_objects={'dice_coef': dice_coef,
                                  'precision':precision,
                                  'recall':recall})

# ### get separable unet model
# sep_unet = Separable_Unet2D(n_filters=64, input_dim_x=input_dim_x, input_dim_y=input_dim_y, num_channels=3)
# model, model_name = sep_unet.get_sep_unet_v2()
# model = load_model('./azh_wound_care_center_diabetic_foot_training_history/' + weight_file_name
#                , custom_objects={'dice_coef': dice_coef,
#                                  'relu6':relu6,
#                                  'DepthwiseConv2D':DepthwiseConv2D,
#                                  'BilinearUpsampling':BilinearUpsampling})

# ### get VGG16 model
# model, model_name = FCN_Vgg16_16s(input_shape=(input_dim_x, input_dim_y, 3))
# with CustomObjectScope({'BilinearUpSampling2D':BilinearUpSampling2D}):
#     model = load_model('./azh_wound_care_center_diabetic_foot_training_history/' + weight_file_name
#                    , custom_objects={'dice_coef': dice_coef})

# ### get mobilenetv2 model
#model = Deeplabv3(input_shape=(input_dim_x, input_dim_y, 3), classes=1)
#model = load_model('./azh_wound_care_center_diabetic_foot_training_history/' + weight_file_name
#               , custom_objects={'dice_coef': dice_coef,
#                                 'relu6':relu6,
#                                 'DepthwiseConv2D':DepthwiseConv2D,
#                                 'BilinearUpsampling':BilinearUpsampling})

# ### get segnet model
######### SegNet ##########
#segnet = SegNet(n_filters=32, input_dim_x=input_dim_x, input_dim_y=input_dim_y, num_channels=3)
#model, model_name = segnet.get_SegNet()
#model = load_model('./merged_azh_dog1_8_history/' + weight_file_name,
#                   custom_objects={'dice_coef': dice_coef,
#                                  'precision':precision,
#                                  'recall':recall})

In [18]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None, None,  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, None, None, 3 896         input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, None, None, 3 9248        conv2d[0][0]                     
__________________________________________________________________________________________________
max_pooling2d (MaxPooling2D)    (None, None, None, 3 0           conv2d_1[0][0]                   
______________________________________________________________________________________________

In [19]:
# Generate predicted mask images
prediction = model.predict(x_test, verbose=1)
#save_results(prediction, 'rgb', path + 'predictions/' + pred_save_path, test_label_filenames_list)

17/17 [==============================] - 1s 55ms/step


## Count the wound area

In [20]:
# Create the pandas DataFrame to store wound area results
wound_area_df = pd.DataFrame(columns = ['Pixel_Area'], index = test_label_filenames_list) 

for i in range(len(test_label_filenames_list)):
    pred = prediction[i]
    ret, pred_bw = cv2.threshold(pred*255., 127, 255, cv2.THRESH_BINARY) 
    wound_area_df.iloc[i,0] = cv2.countNonZero(pred_bw)
#print(wound_area_df)

## Extract date and dog label information from row name

In [21]:
import re
for i in range(len(wound_area_df.index)):
    wound_area_df.loc[wound_area_df.index[i], 'Day'] = int(wound_area_df.index[i].split('-D')[1].split('-')[0])
    wound_area_df.loc[wound_area_df.index[i], 'Dog_label'] = wound_area_df.index[i].split('-')[1].split('-CON')[0]
print(wound_area_df.index)

Index(['-01-CON-D00-L.png', '-01-CON-D02-L.png', '-01-CON-D04-L.png',
       '-01-CON-D07-L.png', '-01-CON-D09-L.png', '-01-CON-D11-L.png',
       '-01-CON-D14-L.png', '-01-CON-D16-L.png', '-01-CON-D18-L.png',
       '-01-CON-D21-L.png',
       ...
       'rotate90-10-CON-D04-R.png', 'rotate90-10-CON-D07-R.png',
       'rotate90-10-CON-D09-R.png', 'rotate90-10-CON-D11-R.png',
       'rotate90-10-CON-D14-R.png', 'rotate90-10-CON-D16-R.png',
       'rotate90-10-CON-D18-R.png', 'rotate90-10-CON-D21-R.png',
       'rotate90-10-CON-D23-R.png', 'rotate90-10-CON-D25-R.png'],
      dtype='object', length=544)


## Find the ratio by calculate the pixel length of 1cm ruler on image

Currently just manually count pixel length of 1 cm ruler
Alternatives: 
1. Hough transform for line detection
2. A separate NN for ruler detection, and count the 1 cm ruler area

In [22]:
# Dog 9 and 10 ration manually picked
#wound_area_df.loc[:,'Ratio'] = [56, 59, 52, 50, 60, 61, 59, 63, 61, 64, 71, 73, 74, 114,
#                             50, 54, 54, 57, 57, 59, 50, 68, 81, 70, 68, 59]

# All dog wound images' ratio, could also read in from file wound_image_labels.csv
wound_area_df.loc[:,'Ratio'] = [35,66,53,54,77,70,62,72,71,71,89,81,65,68,69,30,48,60,66,65,
66,66,64,74,57,69,75,73,74,82,35,56,51,62,69,74,79,90,73,66,73,72,69,74,52,33,40,52,58,66,69,
48,69,68,55,62,64,79,83,64,50,58,64,75,75,96,70,75,63,72,51,48,53,45,53,53,54,50,55,54,67,72,
78,69,60,51,50,57,53,60,53,55,70,71,80,91,86,70,55,57,52,65,60,69,62,75,80,76,80,82,56,59,52,
50,60,61,59,63,61,64,71,73,74,114,50,54,54,57,57,59,50,68,81,70,68,59]*4

wound_area_df.loc[:,'Area_mm2'] = wound_area_df.loc[:,'Pixel_Area']*100 / (wound_area_df.loc[:,'Ratio']**2)
#print(wound_area_df)
wound_area_df = wound_area_df[['Day', 'Dog_label', 'Area_mm2']]

In [23]:
print(wound_area_df[wound_area_df.Dog_label == "07"])

                             Day Dog_label Area_mm2
-07-CON-D00-R.png            0.0        07   146.12
-07-CON-D02-R.png            2.0        07      500
-07-CON-D04-R.png            4.0        07  425.632
-07-CON-D07-R.png            7.0        07  446.528
-07-CON-D09-R.png            9.0        07  252.011
-07-CON-D11-R.png           11.0        07  153.818
-07-CON-D14-R.png           14.0        07  87.5102
-07-CON-D16-R.png           16.0        07  67.4073
-07-CON-D18-R.png           18.0        07  41.4688
-07-CON-D21-R.png           21.0        07   33.124
-07-CON-D23-R.png           23.0        07  10.1947
-07-CON-D25-R.png           25.0        07  20.9796
rotate180-07-CON-D00-R.png   0.0        07    147.6
rotate180-07-CON-D02-R.png   2.0        07  503.478
rotate180-07-CON-D04-R.png   4.0        07  428.622
rotate180-07-CON-D07-R.png   7.0        07  447.556
rotate180-07-CON-D09-R.png   9.0        07  255.927
rotate180-07-CON-D11-R.png  11.0        07  158.347
rotate180-07

## Extract the intermediate layer as features

In [24]:
# Extract the intermediate layer as features
########### SegNet ####################################
# !check the summary to confirm which layer to extract
#
# Segnet: use 'conv2d_5'
# Unet: use 'conv2d_12'

layer_name = 'conv2d_12'
#layer_name = 'conv2d_5'
intermediate_layer_model = keras.Model(inputs=model.input,
                                       outputs=model.get_layer(layer_name).output)
intermediate_output = intermediate_layer_model.predict(x_test)
print(intermediate_output.shape)
# flatten the extracted features using the 1-3 column
intermediate_output_flatten = intermediate_output.reshape((intermediate_output.shape[0],
                                                           intermediate_output.shape[1] * 
                                                           intermediate_output.shape[2] * 
                                                           intermediate_output.shape[3]))
print(intermediate_output_flatten.shape)
print("Are there any NA value? : ", np.any(np.isnan(intermediate_output_flatten)))

(544, 14, 14, 256)
(544, 50176)
Are there any NA value? :  False


In [25]:
# Use PCA to reduce dimension to 50 for the regression model

# change to a pandas dataframe
og_data = pd.DataFrame(intermediate_output_flatten, index = test_label_filenames_list)
#og_data.shape
#print(og_data)

encoded_file_path = path + 'extracted_features/' + pred_save_path
# pca compressed: pca_compressed_features.csv
# vae compressed: VAE-1_compressed_features.csv
#og_data.to_csv(encoded_file_path + 'nocompression_features_rotations.csv', sep='\t')

# Data normalization

#minmax data transformation
from sklearn import preprocessing
#built up data frame
from pandas import DataFrame, Series
norm_data = og_data

# Scale RNAseq data using zero-one normalization
norm_data_zerone = preprocessing.MinMaxScaler().fit_transform(norm_data)

# If select the minmax method
norm_data = pd.DataFrame(norm_data_zerone)
norm_data.head(3)

,0,1,2,3,4,5,6,7,8,9,...,50166,50167,50168,50169,50170,50171,50172,50173,50174,50175
0,0.013010,0.0,0.0,0.0,0.013170,0.022925,0.015403,0.0,0.000000,0.037479,...,0.0,0.0,0.0,0.000000,0.000000,0.029523,0.0,0.0,0.000000,0.019725
1,0.250364,0.0,0.0,0.0,0.869832,0.720803,0.841313,0.0,0.392639,0.737215,...,0.0,0.0,0.0,0.000000,0.292399,0.315715,0.0,0.0,0.187250,0.081713
2,0.000000,0.0,0.0,0.0,0.149114,0.000000,0.187941,0.0,0.041373,0.015389,...,0.0,0.0,0.0,0.059322,0.339779,0.091486,0.0,0.0,0.150419,0.100111


## PCA Projection

In [26]:
# PCA Projection using Minka's MLE
#from sklearn.decomposition import PCA
# use this, if selecting the amount of variance that needs to be explained is greater than the percentage specified by n_components.
# or assign a certain components number, e.g. 50
#pca = PCA(n_components = 0.95, svd_solver = 'full')
#pca = PCA(n_components = 50, svd_solver = 'auto')
#print(pca)
#principalComponents = pca.fit_transform(norm_data)
#principalDf = pd.DataFrame(data = principalComponents, index = test_label_filenames_list)
#principalDf.head(5)

In [27]:
#Plotting the Cumulative Summation of the Explained Variance
#plt.figure()
#plt.plot(np.cumsum(pca.explained_variance_ratio_))
#plt.xlabel('Number of Components')
#plt.ylabel('Variance (%)') #for each component
#plt.title('Dataset Explained Variance')
#plt.savefig('PCA_Explained_Variance(thre_0.9).png')
#plt.show()

# VAE compression

# Merge with area/day information and Save the compressed encoding

In [32]:
# For PCA compressed:
#out_df = principalDf.merge(wound_area_df, how='outer', left_index=True, right_index=True)

# For VAE compressed:
path = './data/all_dog_wounds_noAugmentation/extracted_features/'
pred_save_path = '2021-05-05 02:22:24.668040/'
feature_path = path + pred_save_path + "VAE_compressed_features_rotations.csv"

z_df = pd.read_csv(feature_path, sep = "\t", index_col = 0)
#z_df.head(5)

z_df.columns.name = 'sample_id'
#z_df.columns = z_df.columns + 1
out_df = z_df.merge(wound_area_df, how='outer', left_index=True, right_index=True)
out_df.head(5)

,1,2,3,4,5,6,7,8,9,10,...,494,495,496,497,498,499,500,Day,Dog_label,Area_mm2
-01-CON-D00-L.png,-0.050434,-1.390307,0.167816,-0.112367,-0.408194,0.055195,1.282715,-1.565880,-1.857285,1.037553,...,-1.018288,-0.049699,-0.081388,-0.775323,2.378890,-1.778135,-1.023888,0.0,01,207.347
-01-CON-D02-L.png,-2.234004,3.361946,1.755348,-0.714928,-1.265325,-1.433950,0.213965,0.772141,0.246065,-0.364576,...,-0.185017,-0.740973,0.223862,2.914460,-1.631445,0.107731,2.059691,2.0,01,631.221
-01-CON-D04-L.png,2.126452,-0.038723,0.036344,0.993729,1.101541,0.125694,0.785408,1.228419,-0.016259,0.324081,...,0.706419,1.035234,1.940640,2.493480,1.637801,-0.182631,1.013990,4.0,01,633.001
-01-CON-D07-L.png,0.859735,1.653606,1.450728,0.229802,-0.045004,-0.519277,1.149385,1.813369,0.590547,-0.027623,...,1.171461,-0.504007,0.661376,-0.886728,1.582719,-1.052255,-1.155560,7.0,01,533.711
-01-CON-D09-L.png,0.641620,1.437227,2.577445,-1.141101,0.791560,0.259664,0.645715,-1.143304,0.230593,-1.653891,...,-1.337285,0.036678,1.721318,1.526679,-1.011198,1.057316,-1.024662,9.0,01,463.94


In [34]:
encoded_file_path = path + pred_save_path
# pca compressed: pca_compressed_features.csv
# vae compressed: VAE-1_compressed_features.csv
out_df.to_csv(encoded_file_path + 'VAE_compressed_features_area_rotations.csv', sep='\t')

In [ ]:
#print(path + 'predictions/' + pred_save_path)

In [ ]:
#for image_batch, label_batch in data_gen.generate_data(batch_size=len(x_test), test=True):
#    prediction = model.predict(image_batch, verbose=1)
    #save_results(prediction, 'rgb', path + 'test/predictions/' + pred_save_path, test_label_filenames_list)
#    break